### PyTorch Optimizing Model Parameters
#### LINK: https://pytorch.org/tutorials/beginner/basics/optimization_tutorial.html

In [18]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda
import torchvision.models as models

In [16]:
training_data = datasets.FashionMNIST(
    root="/home/irikos/Work/datasets/pytorch_tutorials",
    train=True,
    download=True,
    transform=ToTensor(),
)

test_data = datasets.FashionMNIST(
    root="/home/irikos/Work/datasets/pytorch_tutorials",
    train=False,
    download=True,
    transform=ToTensor(),
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

In [8]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )
        
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [10]:
model = NeuralNetwork()

In [5]:
# hyperparameters
learning_rate = 1e-3
batch_size = 64
epochs = 5

In [6]:
loss_fn = nn.CrossEntropyLoss()

In [11]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [14]:
### Training loo optimization
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)
        
        # Backpropagation
        optimizer.zero_grad() # the gradients add by default so we set them to zero
        loss.backward()
        optimizer.step()
        
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")
            
    
def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0
    
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
        
        

In [17]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.165709 [    0/60000]
loss: 2.157559 [ 6400/60000]
loss: 2.101970 [12800/60000]
loss: 2.116970 [19200/60000]
loss: 2.091734 [25600/60000]
loss: 1.998995 [32000/60000]
loss: 2.045180 [38400/60000]
loss: 1.964462 [44800/60000]
loss: 1.967949 [51200/60000]
loss: 1.881938 [57600/60000]
Test Error: 
 Accuracy: 55.0%, Avg loss: 1.897428 

Epoch 2
-------------------------------
loss: 1.931899 [    0/60000]
loss: 1.898714 [ 6400/60000]
loss: 1.789252 [12800/60000]
loss: 1.821449 [19200/60000]
loss: 1.736835 [25600/60000]
loss: 1.658980 [32000/60000]
loss: 1.691312 [38400/60000]
loss: 1.592858 [44800/60000]
loss: 1.614430 [51200/60000]
loss: 1.495952 [57600/60000]
Test Error: 
 Accuracy: 62.0%, Avg loss: 1.532608 

Epoch 3
-------------------------------
loss: 1.601478 [    0/60000]
loss: 1.560437 [ 6400/60000]
loss: 1.419829 [12800/60000]
loss: 1.482779 [19200/60000]
loss: 1.382130 [25600/60000]
loss: 1.359491 [32000/60000]
loss: 1.378286 [38400/

### PyTorch Optimizing Model Parameters
#### LINK: https://pytorch.org/tutorials/beginner/basics/saveloadrun_tutorial.html

In [21]:
# save model weights
model_vgg = models.vgg16(pretrained=True)
torch.save(model_vgg.state_dict(), 'model_weights.pth')

In [23]:
# load model weights
model_vgg_load = models.vgg16() # we do not specify pretrained, because we do not load any default weights
model_vgg_load.load_state_dict(torch.load('model_weights.pth'))
model_vgg_load.eval()

# note: be sure to call model.eval() method before inferencing to set the dropout and batch normalization layers 
# to evaluation mode. Failing to do this will yield inconsistent inference results.

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [24]:
# Saving and Loading Models with Shapes
# saves the structure of this class together with the model
torch.save(model, 'model.pth')

# loads the model
model = torch.load('model.pth')